In [ ]:
%qtconsole

In [ ]:
import numpy as np
import pandas as pd
from scipy.fftpack import fft, ifft, fftfreq
import scipy.signal as signal
import matplotlib.pyplot as plt
from configparser import ConfigParser
dB = lambda x: 20*np.log10(np.abs(x))
calibration = 3.55e-3 # mV/Ps
dBuSPL = lambda x: 20*np.log10(np.abs(x)/(20e-6*calibration))
dBu2SPL = lambda x: 10*np.log10(np.abs(x)/(20e-6*calibration)**2)

In [ ]:
folder = 'Two tones high load'
folder = '../Measurements/' + folder + '/'
config = ConfigParser()
config.read(folder + 'settings.ini')

calibration = 3.55e-3 # mV/Ps
timedata=pd.read_csv(folder + 'F1.txt',delimiter='\t')
n=len(timedata)
fs=float(config['General']['fs'])

spectrogram=timedata.apply(lambda x: signal.spectrogram(x,fs=fs,nperseg=2**15,scaling='spectrum'))
frequencydata=timedata.apply(fft,raw=True)/n
f=fftfreq(n,1/fs)

In [ ]:
spectrIdx='Modulated signal'
plt.pcolormesh(spectrogram[spectrIdx][1],
               spectrogram[spectrIdx][0],
               db(spectrogram[spectrIdx][2]),
              vmin=0,vmax=None)
plt.ylim((30e3,45e3))
plt.colorbar()
plt.show()

In [ ]:
spectrIdx='Amplifier'
fMin = 38e3
fMax = 42e3
fIdx = (spectrogram[spectrIdx][0]>fMin) & (spectrogram[spectrIdx][0]<fMax) 
plt.pcolormesh(spectrogram[spectrIdx][1],
               spectrogram[spectrIdx][0][fIdx],
               dB(spectrogram[spectrIdx][2][fIdx]),
               vmin=-100,vmax=-60)
              
plt.ylim((fMin,fMax))
#plt.xlim(xmin=4.4,xmax=4.6)
plt.colorbar()
plt.show()

In [ ]:
spectrIdx='Microphone'
fMin = 71e3
fMax = 73e3
fIdx = (spectrogram[spectrIdx][0]>fMin) & (spectrogram[spectrIdx][0]<fMax) 
maxVal = np.max(dBu2SPL( spectrogram[spectrIdx][2][fIdx] ))
plt.pcolormesh(spectrogram[spectrIdx][1],
               spectrogram[spectrIdx][0][fIdx],
               dBu2SPL(spectrogram[spectrIdx][2][fIdx]),
               vmin=30,vmax=None)
plt.ylim((fMin,fMax))
#plt.xlim(xmin=4.4,xmax=4.6)
plt.colorbar()
plt.show()
print('Maximum value: {:.1f} dB SPL'.format(maxVal))

In [ ]:
idx='Amplifier'
fMin = 1.9e3
fMax = 2.1e3
fIdx= (f>fMin) & (f<fMax)
plt.plot(f[fIdx],dB(frequencydata[idx][fIdx]))
#plt.xlim((26e3,46e3))
plt.ylim(ymin=-100)
print('Max value: {:.1f}'.format(np.max(dB(frequencydata[idx][fIdx]))))
plt.show()

In [ ]:
idx='Microphone'
fMin = 42.9e3
fMax = 43.1e3
fIdx= (f>fMin) & (f<fMax)
plt.plot(f[fIdx],dBuSPL(frequencydata[idx][fIdx]))
#plt.xlim((26e3,46e3))
plt.ylim(ymin=20)
print('Max value: {:.1f}'.format(np.max(dBuSPL(frequencydata[idx][fIdx]))))
plt.show()